In [ ]:
# Valuation of European Call Options using Fast Fourier Transform
# Models: BSM and M76

#Import necessary packages
import math
import numpy as np
from numpy.fft import fft

#Characteristic Functions

def BSM_characteristic_function(v, x0, T, r, sigma):
    '''  Valuation of European call option in BSM model via
    Lewis (2001) and Carr-Madan (1999)
    --> Fourier-based approach: charcteristic function. '''
    cf_value = np.exp(((x0 / T + r - 0.5 * sigma ** 2) * 1j * v
                - 0.5 * sigma ** 2 * v ** 2) * T)
    return cf_value


def M76_characteristic_function(u, x0, T, r, sigma, lamb, mu, delta):
    ''' Valuation of European call option in M76 model via
    Lewis (2001) Fourier-based approach: characteristic function.

    Parameter definitions see function M76_value_call_INT. '''
    omega = x0 / T + r - 0.5 * sigma ** 2 \
        - lamb * (np.exp(mu + 0.5 * delta ** 2) - 1)
    value = np.exp((1j * u * omega - 0.5 * u ** 2 * sigma ** 2 +
                    lamb * (np.exp(1j * u * mu -
                    u ** 2 * delta ** 2 * 0.5) - 1)) * T)
    return value

def H93_char_func(u, x0, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model via Lewis (2001)
    Fourier-based approach: characteristic function.
    Parameter definitions see function BCC_call_value.'''
    c1 = kappa_v * theta_v
    c2 = -np.sqrt((rho * sigma_v * u * 1j - kappa_v)
            ** 2 - sigma_v ** 2 * (-u * 1j - u ** 2))
    c3 = (kappa_v - rho * sigma_v * u * 1j + c2) \
          / (kappa_v - rho * sigma_v * u * 1j - c2)
    H1 = (r * u * 1j * T + (c1 / sigma_v ** 2)
          * ((kappa_v - rho * sigma_v * u * 1j + c2) * T
                - 2 * np.log((1 - c3 * np.exp(c2 * T)) / (1 - c3))))
    H2 = ((kappa_v - rho * sigma_v * u * 1j + c2) / sigma_v ** 2
          * ((1 - np.exp(c2 * T)) / (1 - c3 * np.exp(c2 * T))))
    char_func_value = np.exp(H1 + H2 * v0)
    return char_func_value

def BCC_char_func(u, x0, T, r, kappa_v, theta_v, sigma_v, rho, v0,
                    lamb, mu, delta):
    ''' Valuation of European call option in BCC97 model via Lewis (2001)
    Fourier-based approach: characteristic function.
    Parameter definitions see function BCC_call_value.'''
    BCC1 = H93_char_func(u, x0, T, r, kappa_v, theta_v, sigma_v, rho, v0)
    BCC2 = M76_char_func(u, x0, T, lamb, mu, delta)
    return BCC1 * BCC2

#Valuation by FFT for BSM model

def BSM_call_value_FFT(S0, K, T, r, sigma):
    ''' Valuation of European call option in BSM model via Lewis (2001)
    --> Fourier-based approach (integral).
    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    sigma: float
        volatility factor in diffusion term
    Returns
    =======
    call_value: float
        European call option present value
    '''
    k = np.log(K / S0)
    x0 = np.log(S0 / S0)
    g = 1  # factor to increase accuracy
    N = g * 4096
    eps = (g * 150.) ** -1
    eta = 2 * np.pi / (N * eps)
    b = 0.5 * N * eps - k
    u = np.arange(1, N + 1, 1)
    
    v1 = -N/2*eta+eta/2
    vN = N/2*eta-eta/2
    k1 = -b
    
    vu = v1 + eta * (u - 1)
    ku = k
    v = -vu-0.5*1j
    
    # Numerical FFT Routine
    delt = np.zeros(N, dtype=np.float)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    SimpsonW = (3 + (-1) ** j - delt) / 3
    FFTFunc = (np.exp(-1j * k1 * (u-1)*eta) * BSM_characteristic_function(v, x0, T, r, sigma)/(vu**2+0.25)) * SimpsonW
    payoff = (np.exp(-1j * v1 * k) * eta* fft(FFTFunc)).real
    CallValueM = S0-np.sqrt(S0*K)*np.exp(-r*T)*payoff / (2*np.pi)
    pos = int((k+b) / eps)
    CallValue = CallValueM[pos]
    # klist = np.exp((np.arange(0, N, 1) - 1) * eps - b) * S0
    #CallValue =  interp1d(ku,CallValueM)
    return CallValue #, klist[pos - 50:pos + 50]


#Valuation by FFT for M76 model

def M76_call_value_FFT(S0, K, T, r, sigma, lamb, mu, delta):
    ''' Valuation of European call option in M76 model via
    Carr-Madan (1999) Fourier-based approach.

    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    sigma: float
        volatility factor in diffusion term
    lamb: float
        jump intensity
    mu: float
        expected jump size
    delta: float
        standard deviation of jump

    Returns
    =======
    call_value: float
        European call option present value
    '''
    k = math.log(K / S0)
    x0 = math.log(S0 / S0)
    g = 2  # factor to increase accuracy
    N = g * 4096
    eps = (g * 150.) ** -1
    eta = 2 * math.pi / (N * eps)
    b = 0.5 * N * eps - k
    u = np.arange(1, N + 1, 1)
    
    v1 = -N/2*eta+eta/2
    vN = N/2*eta-eta/2
    k1 = -b
    
    vu = v1 + eta * (u - 1)
    ku = k
    v = -vu-0.5*1j
    
    # Numerical FFT Routine
    delt = np.zeros(N, dtype=np.float)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    SimpsonW = (3 + (-1) ** j - delt) / 3
    FFTFunc = (np.exp(-1j * k1 * (u-1)*eta) * M76_characteristic_function(v, x0, T, r, sigma, lamb, mu, delta)/(vu**2+0.25)) * SimpsonW
    payoff = (np.exp(-1j * v1 * k) * eta* fft(FFTFunc)).real
    CallValueM = S0-np.sqrt(S0*K)*np.exp(-r*T)*payoff / (2*np.pi)
    pos = int((k+b) / eps)
    CallValue = CallValueM[pos]
    # klist = np.exp((np.arange(0, N, 1) - 1) * eps - b) * S0
    #CallValue =  interp1d(ku,CallValueM)
    return CallValue #, klist[pos - 50:pos + 50]
                                        
def H93_call_value_FFT(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0):
    ''' Valuation of European call option in H93 model via
    Carr-Madan (1999) Fourier-based approach.

    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial level of variance

    Returns
    =======
    call_value: float
        European call option present value
    '''
    k = math.log(K / S0)
    x0 = math.log(S0 / S0)
    g = 2  # factor to increase accuracy
    N = g * 4096
    eps = (g * 150.) ** -1
    eta = 2 * math.pi / (N * eps)
    b = 0.5 * N * eps - k
    u = np.arange(1, N + 1, 1)
    
    v1 = -N/2*eta+eta/2
    vN = N/2*eta-eta/2
    k1 = -b
    
    vu = v1 + eta * (u - 1)
    ku = k
    v = -vu-0.5*1j
    
    # Numerical FFT Routine
    delt = np.zeros(N, dtype=np.float)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    SimpsonW = (3 + (-1) ** j - delt) / 3
    FFTFunc = (np.exp(-1j * k1 * (u-1)*eta) * H93_char_func(v, x0, T, r, kappa_v, theta_v, sigma_v, rho, v0)/(vu**2+0.25)) * SimpsonW
    payoff = (np.exp(-1j * v1 * k) * eta* fft(FFTFunc)).real
    CallValueM = S0-np.sqrt(S0*K)*np.exp(-r*T)*payoff / (2*np.pi)
    pos = int((k+b) / eps)
    CallValue = CallValueM[pos]
    # klist = np.exp((np.arange(0, N, 1) - 1) * eps - b) * S0
    #CallValue =  interp1d(ku,CallValueM)
    return CallValue #, klist[pos - 50:pos + 50]


def BCC_call_value_FFT(S0, K, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta):
    ''' Valuation of European call option in BCC97 model via
    Carr-Madan (1999) Fourier-based approach.

    Parameters
    ==========
    S0: float
        initial stock/index level
    K: float
        strike price
    T: float
        time-to-maturity (for t=0)
    r: float
        constant risk-free short rate
    kappa_v: float
        mean-reversion factor
    theta_v: float
        long-run mean of variance
    sigma_v: float
        volatility of variance
    rho: float
        correlation between variance and stock/index level
    v0: float
        initial level of variance
    lamb: float
        jump intensity
    mu: float
        expected jump size
    delta: float
        standard deviation of jump
    Returns
    =======
    call_value: float
        European call option present value
    '''
    k = math.log(K / S0)
    x0 = math.log(S0 / S0)
    g = 3  # factor to increase accuracy
    N = g * 4096
    eps = (g * 150.) ** -1
    eta = 2 * math.pi / (N * eps)
    b = 0.5 * N * eps - k
    u = np.arange(1, N + 1, 1)
    
    v1 = -N/2*eta+eta/2
    vN = N/2*eta-eta/2
    k1 = -b
    
    vu = v1 + eta * (u - 1)
    ku = k
    v = -vu-0.5*1j
    
    # Numerical FFT Routine
    delt = np.zeros(N, dtype=np.float)
    delt[0] = 1
    j = np.arange(1, N + 1, 1)
    SimpsonW = (3 + (-1) ** j - delt) / 3
    FFTFunc = (np.exp(-1j * k1 * (u-1)*eta) * BCC_char_func(v, x0, T, r, kappa_v, theta_v, sigma_v, rho, v0, lamb, mu, delta)/(vu**2+0.25)) * SimpsonW
    payoff = (np.exp(-1j * v1 * k) * eta* fft(FFTFunc)).real
    CallValueM = S0-np.sqrt(S0*K)*np.exp(-r*T)*payoff / (2*np.pi)
    pos = int((k+b) / eps)
    CallValue = CallValueM[pos]
    # klist = np.exp((np.arange(0, N, 1) - 1) * eps - b) * S0
    #CallValue =  interp1d(ku,CallValueM)
    return CallValue #, klist[pos - 50:pos + 50]

    